In [22]:
import pandas as pd
import warnings
import glob
import os
warnings.filterwarnings("ignore")

In [48]:
def compute_hit_rate_ethnicity(path, county, city):    
    df = pd.read_csv(path, compression = 'zip')
    df_searched = df.query("search_conducted == True")
    df_searched["date"] = pd.to_datetime(df_searched["date"])
    df_searched["year"] = df_searched["date"].dt.year
    df_searched_cleaned = df_searched[['year', 'subject_race', 'subject_sex', 'search_conducted', 'contraband_found']]
    found = pd.DataFrame(df_searched_cleaned.groupby(['year', 'subject_race'])['contraband_found'].value_counts()).rename(columns={'contraband_found' : 'nb_find'}).reset_index()
    all = pd.DataFrame(df_searched_cleaned.groupby(['year', 'subject_race'])['search_conducted'].value_counts()).rename(columns={'search_conducted' : 'nb_search'}).reset_index()
    found_and_all = pd.merge(left = found, right = all, left_on=['year','subject_race'], right_on = ['year', 'subject_race']).drop(['search_conducted'], axis = 1)
    found_and_all['Hit_rate'] = found_and_all['nb_find'] / found_and_all['nb_search'] * 100
    found_and_all['County'] = county
    found_and_all['City'] = city
    return found_and_all.query('contraband_found == True')

In [50]:
all_city = compute_hit_rate_ethnicity('..\data\city\ca_oakland.csv.zip', 'California', 'Oakland')

Done on all the file

In [69]:
new_df = compute_hit_rate_ethnicity('../data/city/tx_san_antonio.csv.zip', 'Texas', 'San Antonio')

In [70]:
all_city = pd.concat([all_city, new_df])

In [72]:
all_city.to_csv('../data/city/all_hit_rate_ethnicity')